In [12]:
let mut a: u32 = 10;
println!("{a}");
a = 20;
println!("{a}");

10
20


# Pattern Matching

## Let Control Flow

### if-let

In [3]:
fn sleep_for(secs: f32) {
    let dur = if let Ok(dur_) = std::time::Duration::try_from_secs_f32(secs) {
        dur_
    } else {
        std::time::Duration::from_millis(500)
    };
    std::thread::sleep(dur);
    println!("slept for {:?}", dur);
}

In [4]:
sleep_for(-10.0);
sleep_for(0.8);
sleep_for(0.5);

slept for 500ms
slept for 800.000012ms
slept for 500ms


### let-else

In [5]:
#[derive(Debug)]
enum Result {
    Ok(u32),
    Err(String),
}

fn hex_or_die_trying(maybe_string: Option<String>) -> Result {
    let s = if let Some(s) = maybe_string {
        s
    } else {
        return Result::Err(String::from("got None"));
    };

    let first_byte_char = if let Some(first_byte_char) = s.chars().next() {
        first_byte_char
    } else {
        return Result::Err(String::from("got empty string"));
    };

    if let Some(digit) = first_byte_char.to_digit(16) {
        Result::Ok(digit)
    } else {
        Result::Err(String::from("not a hex digit"))
    }
}

In [6]:
println!("result: {:?}", hex_or_die_trying(Some(String::from("foo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from("Aoo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from(""))));
println!("result: {:?}", hex_or_die_trying(None));

result: Ok(15)
result: Ok(10)
result: Err("got empty string")
result: Err("got None")


#### neat version

In [7]:
fn hex_or_die_trying_flatten(maybe_string: Option<String>) -> Result {
    let Some(s) = maybe_string else {
        return Result::Err(String::from("got None"));
    };

    let Some(first_byte_char) = s.chars().next() else {
        return Result::Err(String::from("got empty string"));
    };

    let Some(digit) = first_byte_char.to_digit(16) else {
        return Result::Err(String::from("not a hex digit"));
    };

    Result::Ok(digit)
}

In [8]:
println!("result: {:?}", hex_or_die_trying(Some(String::from("foo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from("Aoo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from(""))));
println!("result: {:?}", hex_or_die_trying(None));

result: Ok(15)
result: Ok(10)
result: Err("got empty string")
result: Err("got None")


### while-let

In [9]:
let mut name = String::from("Comprehensive Rust 🦀");
while let Some(c) = name.pop() {
    println!("character: {c}");
}

character: 🦀
character:  
character: t
character: s
character: u
character: R
character:  
character: e
character: v
character: i
character: s
character: n
character: e
character: h
character: e
character: r
character: p
character: m
character: o
character: C


()

# Methods and Traits

## Methods

In [10]:
#[derive(Debug)]
struct Race {
    name: String,
    laps: Vec<i32>,
}

impl Race {
    // No receiver, a static method
    fn new(name: &str) -> Self {
        Self { name: String::from(name), laps: Vec::new() }
    }

    // Exclusive borrowed read-write access to self
    fn add_lap(&mut self, lap: i32) {
        self.laps.push(lap);
    }

    // Shared and read-only borrowed access to self
    fn print_laps(&self, index: bool) {
        println!("Recorded {} laps for {}:", self.laps.len(), self.name);
        if index {
            for (idx, lap) in self.laps.iter().enumerate() {
                println!("Lap {idx}: {lap} sec");
            }
        } else {
            for lap in self.laps.iter() {
                println!("{lap} sec");
            }
        }
    }

    // Exclusive ownership of self
    fn finish(self) {
        let total: i32 = self.laps.iter().sum();
        println!("Race {} is finished, total lap time: {}", self.name, total);
    }
}

In [11]:
let mut race = Race::new("Monaco Grand Prix");
race.add_lap(70);
race.add_lap(68);
race.print_laps(false);
race.add_lap(71);
race.print_laps(true);
race.finish();

Recorded 2 laps for Monaco Grand Prix:
70 sec
68 sec
Recorded 3 laps for Monaco Grand Prix:
Lap 0: 70 sec
Lap 1: 68 sec
Lap 2: 71 sec
Race Monaco Grand Prix is finished, total lap time: 209


## Traits

In [2]:
struct Dog {
    name: String,
    age: i8,
}
struct Cat {
    lives: i8,
}

trait Pet {
    fn talk(&self) -> String;

    fn greet(&self) {
        println!("Oh you're a cutie! What's your name? {}", self.talk());
    }
}

impl Pet for Dog {
    fn talk(&self) -> String {
        format!("Woof, my name is {}!", self.name)
    }
}

impl Pet for Cat {
    fn talk(&self) -> String {
        String::from("Miau!")
    }
}

In [3]:
let captain_floof = Cat { lives: 9 };
let fido = Dog { name: String::from("Fido"), age: 5 };

captain_floof.greet();
fido.greet();

Oh you're a cutie! What's your name? Miau!
Oh you're a cutie! What's your name? Woof, my name is Fido!


## Deriving

In [4]:
#[derive(Debug, Clone, Default)]
struct Player {
    name: String,
    strength: u8,
    hit_points: u8,
}

In [5]:
let p1 = Player::default(); // Default trait adds `default` constructor.
let mut p2 = p1.clone(); // Clone trait adds `clone` method.
p2.name = String::from("EldurScrollz");
// Debug trait adds support for printing with `{:?}`.
println!("{:?} vs. {:?}", p1, p2);

Player { name: "", strength: 0, hit_points: 0 } vs. Player { name: "EldurScrollz", strength: 0, hit_points: 0 }


## Trait Objects

In [6]:
struct Dog {
    name: String,
    age: i8,
}
struct Cat {
    lives: i8,
}

trait Pet {
    fn talk(&self) -> String;
}

impl Pet for Dog {
    fn talk(&self) -> String {
        format!("Woof, my name is {}!", self.name)
    }
}

impl Pet for Cat {
    fn talk(&self) -> String {
        String::from("Miau!")
    }
}

In [12]:
let pets: Vec<Box<dyn Pet>> = vec![
    Box::new(Cat { lives: 9 }),
    Box::new(Dog { name: String::from("Fido"), age: 5 }),
];
for pet in pets {
    println!("Hello, who are you? {}", pet.talk());
}

Hello, who are you? Miau!
Hello, who are you? Woof, my name is Fido!


()

In [13]:
println!("{} {}", std::mem::size_of::<Dog>(), std::mem::size_of::<Cat>());
println!("{} {}", std::mem::size_of::<&Dog>(), std::mem::size_of::<&Cat>());
println!("{}", std::mem::size_of::<&dyn Pet>());
println!("{}", std::mem::size_of::<Box<dyn Pet>>());

32 1
8 8
16
16


# Generic

## Generic Functions

In [40]:
/// Pick `even` or `odd` depending on the value of `n`.
fn pick<T>(n: i32, even: T, odd: T) -> T {
    if n % 2 == 0 {
        even
    } else {
        odd
    }
}

println!("picked a number: {:?}", pick(97, 222, 333));
println!("picked a tuple: {:?}", pick(28, ("dog", 1), ("cat", 2)));

picked a number: 333
picked a tuple: ("dog", 1)


## Generic Data Types

In [41]:
#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

impl Point<u32> {
    fn coords(&self) -> (&u32, &u32) {
        (&self.x, &self.y)
    }

    // fn set_x(&mut self, x: T)
}

impl Point<f64> {
    fn coords(&self) -> (&f64, u32, &f64) {
        (&self.x, 0, &self.y)
    }
}

// impl<T> Point<T> {
//     fn coords(&self) -> &T {
//         &self.x + &self.y
//     }
// }

let integer = Point { x: 5, y: 10 };
let float = Point { x: 1.0, y: 4.0 };
println!("{integer:?} and {float:?}");
println!("coords(u32): {:?}", integer.coords());
println!("coords(f64): {:?}", float.coords());

Point { x: 5, y: 10 } and Point { x: 1.0, y: 4.0 }
coords(u32): (5, 10)
coords(f64): (1.0, 0, 4.0)


In [ ]:
#[derive(Debug)]
struct Point<T, U> {
    x: T,
    y: U,
}

impl<T, U> Point<T, U> {
    fn coords(&self) -> (&T, &U) {
        (&self.x, &self.y)
    }
}

let mix = Point { x: 2, y: 3.0 };
println!("{mix:?}");
println!("coords: {:?}", mix.coords());

## Trait Bounds

In [20]:
fn duplicate<T: Clone>(a: T) -> (T, T) {
    (a.clone(), a.clone())
}

#[derive(Debug)]
#[derive(Clone)]
struct NotClonable {
    name: String,
    age: i32,
}

let foo = String::from("foo");
let foo_pair = duplicate(foo);
println!("{foo_pair:?}");

let me = NotClonable { name: String::from("Ethan"), age: 23 };
let me_pair = duplicate(me);
println!("{bar_pair:?}");

("foo", "foo")
(NotClonable { name: "Ethan", age: 23 }, NotClonable { name: "Ethan", age: 23 })


## impl Trait

In [23]:
// Syntactic sugar for:
//   fn add_42_millions<T: Into<i32>>(x: T) -> i32 {
fn add_42_millions(x: impl Into<i32>) -> i32 {
    x.into() + 42_000_000
}

fn pair_of(x: u32) -> impl std::fmt::Debug {
    (x + 1, x - 1)
}
    

let many = add_42_millions(42_i8);
println!("{many}");
let many_more = add_42_millions(10_000_000);
println!("{many_more}");
// let debuggable = pair_of(27);
// println!("debuggable: {debuggable:?}");

42000042
52000000


# Standard Library Types

## Option

In [32]:
let name = "Löwe 老虎 Léopard Gepardi";
let mut position: Option<usize> = name.find('é');
println!("find returned {position:?}");
assert_eq!(position.unwrap(), 14);
position = name.find('Z');
println!("find returned {position:?}");
assert_eq!(position.expect("Character not found"), 0);

find returned Some(14)
find returned None


thread '<unnamed>' panicked at src/lib.rs:143:21:
Character not found
stack backtrace:
   0: rust_begin_unwind
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/std/src/panicking.rs:645:5
   1: core::panicking::panic_fmt
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/core/src/panicking.rs:72:14
   2: core::panicking::panic_display
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/core/src/panicking.rs:178:5
   3: core::panicking::panic_str
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/core/src/panicking.rs:152:5
   4: core::option::expect_failed
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/core/src/option.rs:1985:5
   5: <unknown>
   6: <unknown>
   7: evcxr::runtime::Runtime::run_loop
   8: evcxr::runtime::runtime_hook
   9: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Result

In [16]:
use std::fs::File;
use std::io::Write;
use std::io::Read;

let mut file = File::create("diary.txt").unwrap();
file.write_all(b"When did I write something here?\nI don't remember!").expect("failed to write message");

let file: Result<File, std::io::Error> = File::open("diary.txt");
match file {
    Ok(mut file) => {
        let mut contents = String::new();
        if let Ok(bytes) = file.read_to_string(&mut contents) {
            println!("Dear diary: \n{contents} \n({bytes} bytes)");
        } else {
            println!("Could not read file content");
        }
    }
    Err(err) => {
        println!("The diary could not be opened: {err}");
    }
}

Dear diary: 
When did I write something here?
I don't remember! 
(50 bytes)


()

## String

In [42]:
let mut s1 = String::new();
s1.push_str("Hello");
println!("s1: {s1} len = {}, capacity = {}", s1.len(), s1.capacity());

let mut s2 = String::with_capacity(s1.len() + 1);
s2.push_str(&s1);
s2.push('!');
println!("s2: {s2} len = {}, capacity = {}", s2.len(), s2.capacity());
s2.push('!');
println!("s2: {s2} len = {}, capacity = {}", s2.len(), s2.capacity());

let s3 = String::from("🇨🇭");
println!("s3: {s3} len = {}, capacity = {}, number of chars = {}", s3.len(), s3.capacity(), s3.chars().count());
for (i, c) in s3.chars().enumerate() {
    println!("char {i}: {c}");
}

s1: Hello len = 5, capacity = 8
s2: Hello! len = 6, capacity = 6
s2: Hello!! len = 7, capacity = 12
s3: 🇨🇭 len = 8, capacity = 8, number of chars = 2
char 0: 🇨
char 1: 🇭


()

## Vec

In [53]:
let mut v1 = Vec::new();
v1.push(42);
println!("v1: {:?}, len = {}, capacity = {}", v1, v1.len(), v1.capacity());

let mut v2 = Vec::with_capacity(v1.len() + 1);
v2.extend(v1.iter());
v2.push(9999);
println!("v2: {:?}, len = {}, capacity = {}", v2, v2.len(), v2.capacity());
v2.push(1026);
println!("v2: {:?}, len = {}, capacity = {}", v2, v2.len(), v2.capacity());

// Canonical macro to initialize a vector with elements.
let mut v3 = vec![0, 0, 1, 2, 3, 4];

// Retain only the even elements.
v3.retain(|x| x % 2 == 0);
println!("v3: {v3:?}");

// Remove consecutive duplicates.
v3.dedup();
println!("v3: {v3:?}");
v3.push(0);
println!("v3: {v3:?}");
v3.dedup();
println!("v3: {v3:?}");

// Use vector to store UTF-8 encoded strings.
let mut v4 = Vec::new();
v4.extend(String::from("Hello 🦀").chars());
println!("v4: {v4:?}");

v1: [42], len = 1, capacity = 4
v2: [42, 9999], len = 2, capacity = 2
v2: [42, 9999, 1026], len = 3, capacity = 4
v3: [0, 0, 2, 4]
v3: [0, 2, 4]
v3: [0, 2, 4, 0]
v3: [0, 2, 4, 0]
v4: ['H', 'e', 'l', 'l', 'o', ' ', '🦀']


## HashMap

In [61]:
use std::collections::HashMap;

let mut page_counts = HashMap::new();
page_counts.insert("Adventures of Huckleberry Finn".to_string(), 207);
page_counts.insert("Grimms' Fairy Tales".to_string(), 751);
page_counts.insert("Pride and Prejudice".to_string(), 303);

if !page_counts.contains_key("Les Misérables") {
    println!(
        "We know about {} books, but not Les Misérables.",
        page_counts.len()
    );
}

for book in ["Pride and Prejudice", "Alice's Adventure in Wonderland"] {
    match page_counts.get(book) {
        Some(count) => println!("{book}: {count} pages"),
        None => println!("{book} is unknown."),
    }
}

// Use the .entry() method to insert a value if nothing is found.
for book in ["Pride and Prejudice", "Alice's Adventure in Wonderland"] {
    let page_count: &mut i32 = page_counts.entry(book.to_string()).or_insert(0);
    *page_count += 1;
}

{
    let pc1 = page_counts.get("Harry Potter and the Sorcerer's Stone").unwrap_or(&336);
    println!("Harry Potter has {} pages", pc1);
}
println!("page_counts:\n{page_counts:#?}");

We know about 3 books, but not Les Misérables.
Pride and Prejudice: 303 pages
Alice's Adventure in Wonderland is unknown.
Harry Potter has 336 pages
page_counts:
{
    "Alice's Adventure in Wonderland": 1,
    "Grimms' Fairy Tales": 751,
    "Pride and Prejudice": 304,
    "Adventures of Huckleberry Finn": 207,
}


# Standard Library Traits

## Comparisons

### PartialEq and Eq

In [73]:
struct Key {
    id: u32,
    metadata: Option<String>,
}

impl PartialEq for Key {
    fn eq(&self, other: &Self) -> bool {
        self.id == other.id
    }
}

let key1 = Key { id: 5, metadata: None };
let key2 = Key { metadata: Some("color=red".to_string()), ..key1 };

println!("Equal? {}", key1 == key2);

Equal? true


### PartialOrd and Ord

In [80]:
use std::cmp::Ordering;
#[derive(Eq, PartialEq)]
struct Citation {
    author: String,
    year: u32,
}

impl PartialOrd for Citation {
    fn partial_cmp(&self, other: &Self) -> Option<Ordering> {
        match self.author.partial_cmp(&other.author) {
            Some(Ordering::Equal) => self.year.partial_cmp(&other.year),
            author_ord => author_ord,
        }
    }
}

// impl Ord for Citation {
//     fn cmp(&self, other: &Self) -> Ordering {
//         self.partial_cmp(other).unwrap()
//     }
// }

let cite1 = Citation { author: "Einstein".to_string(), year: 1905 };
let cite2 = Citation { author: "Ethan Wang".to_string(), year: 2023 };

println!("cite1 < cite2? {}", cite1 < cite2);

cite1 < cite2? true


## Operators

In [91]:
#[derive(Debug, Copy, Clone)]
struct Point {
    x: i32,
    y: i32,
}

impl std::ops::Add for Point {
    type Output = Self;

    fn add(self, other: Self) -> Self {
        Self { x: self.x + other.x, y: self.y + other.y }
    }
}

let p1 = Point { x: 10, y: 20 };
let p2 = Point { x: 100, y: 200 };
println!("{:?} + {:?} = {:?}", p1, p2, p1 + p2);

impl std::ops::Add<(i32, i32)> for Point {
    type Output = (i32, i32);

    fn add(self, other: (i32, i32)) -> Self::Output {
        (self.x + other.0, self.y + other.1)
    }
}

let p3 = (1, 2);
let p4 = p1 + p3;
println!("{:?} + {:?} = {:?}", p1, p3, p4);

Point { x: 10, y: 20 } + Point { x: 100, y: 200 } = Point { x: 110, y: 220 }
Point { x: 10, y: 20 } + (1, 2) = (11, 22)


## From and Into

In [12]:
// From
let s = String::from("hello");
let addr = std::net::Ipv4Addr::from([127, 0, 0, 1]);
let one = i16::from(true);
let bigger = i32::from(123_i16);
println!("{s}, {addr}, {one}, {bigger}");

hello, 127.0.0.1, 1, 123


In [13]:
// Into
let s: String = "hello".into();
let addr: std::net::Ipv4Addr = [127, 0, 0, 1].into();
let one: i16 = true.into();
let bigger: i32 = 123_i16.into();
println!("{s}, {addr}, {one}, {bigger}");

hello, 127.0.0.1, 1, 123


## Casting

In [4]:
let value: i64 = 1000;
println!("as u16: {}", value as u16);
println!("as i16: {}", value as i16);
println!("as u8: {}", value as u8);

as u16: 1000
as i16: 1000
as u8: 232


## Read and Write

In [14]:
// Read
use std::io::{BufRead, BufReader, Read, Result};

fn count_lines<R: Read>(reader: R) -> usize {
    let buf_reader = BufReader::new(reader);
    buf_reader.lines().count()
}

let slice: &[u8] = b"foo\nbar\nbaz\n";
println!("lines in slice: {}", count_lines(slice));

let file = std::fs::File::open(std::env::current_exe()?)?;
println!("lines in file: {}", count_lines(file));

lines in slice: 3
lines in file: 70131


In [17]:
// Write
use std::io::{Result, Write};

fn log<W: Write>(writer: &mut W, msg: &str) -> Result<()> {
    writer.write_all(msg.as_bytes())?;
    writer.write_all("\n".as_bytes())
}

let mut buffer = Vec::new();
log(&mut buffer, "Hello")?;
log(&mut buffer, "World")?;
println!("Logged: {:?}", buffer);

Logged: [72, 101, 108, 108, 111, 10, 87, 111, 114, 108, 100, 10]


## The Default Trait

In [22]:
#[derive(Debug, Default)]
struct Derived {
    x: u32,
    y: String,
    z: Implemented,
}

#[derive(Debug)]
struct Implemented(String);

impl Default for Implemented {
    fn default() -> Self {
        Self("John Smith".into())
    }
}

let default_struct = Derived::default();
println!("Default struct:\n{:#?}\n", default_struct);

let almost_default_struct =
    Derived { y: "Y is set!".into(), ..Derived::default() };
println!("Default struct (with y set):\n{almost_default_struct:#?}\n");

let nothing: Option<Derived> = None;
println!("Fill Nothing with Default:\n{:#?}\n", nothing.unwrap_or_default());

Default struct:
Derived {
    x: 0,
    y: "",
    z: Implemented(
        "John Smith",
    ),
}

Default struct (with y set):
Derived {
    x: 0,
    y: "Y is set!",
    z: Implemented(
        "John Smith",
    ),
}

Fill Nothing with Default:
Derived {
    x: 0,
    y: "",
    z: Implemented(
        "John Smith",
    ),
}



## Closures

In [29]:
fn apply_with_log(func: impl FnOnce(i32) -> i32, input: i32) -> i32 {
    println!("Calling function on {input}");
    func(input)
}

{
    let add_3 = |x| x + 3;
    println!("add_3: {}", apply_with_log(add_3, 10));
    println!("add_3: {}", apply_with_log(add_3, 20));
    
    let mut v = Vec::new();
    let mut accumulate = |x: i32| {
        v.push(x);
        v.iter().sum::<i32>()
    };
    println!("accumulate: {}", apply_with_log(&mut accumulate, 4));
    println!("accumulate: {}", apply_with_log(&mut accumulate, 5));
    
    let multiply_sum = |x| x * v.into_iter().sum::<i32>();
    println!("multiply_sum: {}", apply_with_log(multiply_sum, 3));
};

Calling function on 10
add_3: 13
Calling function on 20
add_3: 23
Calling function on 4
accumulate: 4
Calling function on 5
accumulate: 9
Calling function on 3
multiply_sum: 27


In [28]:
fn make_greeter(prefix: String) -> impl Fn(&str) {
    return move |name| println!("{} {}", prefix, name);
}

{
    let hi = make_greeter("Hi".to_string());
    hi("there");
};

Hi there


# Memory Management

## Ownership

In [2]:
struct Point(i32, i32);

{
    let p = Point(3, 4);
    println!("x: {}", p.0);
}
println!("y: {}", p.1);

Error: cannot find value `p` in this scope

## Move Semantics 

In [5]:
let s1: String = String::from("Hello!");
let s2: String = s1;
println!("s2: {s2}");
// println!("s1: {s1}");

s2: Hello!


In [16]:
fn say_hello(name: String) {
    println!("Hello {name}")
}

fn say_hello_ref(name: &String) {
    println!("Hello {name}")
}

let name = String::from("Alice");
say_hello_ref(&name);
say_hello_ref(&name);

say_hello(name.clone());
say_hello(name);
// say_hello(name);

Hello Alice
Hello Alice
Hello Alice
Hello Alice


## Clone

In [17]:
#[derive(Default)]
struct Backends {
    hostnames: Vec<String>,
    weights: Vec<f64>,
}

impl Backends {
    fn set_hostnames(&mut self, hostnames: &Vec<String>) {
        self.hostnames = hostnames.clone();
        self.weights = hostnames.iter().map(|_| 1.0).collect();
    }
}

## Copy Types

In [19]:
let x = 42;
let y = x;
println!("x: {x}"); // would not be accessible if not Copy
println!("y: {y}");

x: 42
y: 42


In [26]:
#[derive(Copy, Clone, Debug)]
struct Point(i32, i32);

let p1 = Point(3, 4);
let p2 = p1;
println!("p1: {p1:?}");
println!("p2: {p2:?}");

p1: Point(3, 4)
p2: Point(3, 4)


## Drop

In [36]:
struct Droppable {
    name: &'static str,
}

impl Drop for Droppable {
    fn drop(&mut self) {
        println!("Dropping {}", self.name);
    }
}
{
    let a = Droppable { name: "a" };
    {
        let b = Droppable { name: "b" };
        {
            let c = Droppable { name: "c" };
            let d = Droppable { name: "d" };
            println!("Exiting block B");
        }
        println!("Exiting block A");
    }
    drop(a);
    println!("Exiting main");
}

Exiting block B
Dropping d
Dropping c
Exiting block A
Dropping b
Exiting main
Dropping a


()

# Smart Pointers

## Box

In [2]:
let five = Box::new(5);
println!("five: {}", *five);

five: 5


In [3]:
#[derive(Debug)]
enum List<T> {
    /// A non-empty list: first element and the rest of the list.
    Element(T, Box<List<T>>),
    /// An empty list.
    Nil,
}

let list: List<i32> =
    List::Element(1, Box::new(List::Element(2, Box::new(List::Nil))));
println!("{list:?}");

Element(1, Element(2, Nil))


## RC

In [7]:
use std::rc::Rc;

let a = Rc::new(10);
println!("a: {a}");
println!("a strong: {}", Rc::strong_count(&a));

let b = Rc::clone(&a);

println!("b: {b}");
println!("a strong: {}", Rc::strong_count(&a));
println!("b strong: {}", Rc::strong_count(&b));

a: 10
a strong: 1
b: 10
a strong: 2
b strong: 2
